[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1oXrz0IL9rZwnwHijHWcXQcARkSyUPL_x)


## dash-whales

**To run on Colab use the code below** :
```
git clone https://github.com/mohcineelharras/dash-whales
cd dash-whales
```

## Install necessary libraries
```
!pip install -r requirements.txt
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install ngrok
```


In [18]:
!git clone https://github.com/mohcineelharras/dash-whales
%cd "dash-whales"

Cloning into 'dash-whales'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 44 (delta 17), reused 34 (delta 9), pack-reused 0
Unpacking objects: 100% (44/44), 3.38 MiB | 7.03 MiB/s, done.
/content/dash-whales/dash-whales


In [19]:
!pip install -r requirements.txt
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install ngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Scrap Data from etherscan
same code as in the file scrap_data.py

In [1]:
import requests
import time
import pandas as pd
import json
import os

# Etherscan API URL
url = "https://api.etherscan.io/api"

# Your Etherscan API Key
api_key = "VVJFE7IG5WSK8P2UJIJN5MST5UBQEZYVYW"

# Create output folder
if not os.path.exists("output"):
    os.makedirs("output")

# Load the JSON file into a dictionary
with open("ressources/dict_tokens_addr.json", "r") as file:
    dict_addresses = json.load(file)

def get_coin_data(tokenSymbol, contractAddr, n):

    # Get the latest block number
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    latest_block = int(response.json()["result"], 16)

    # Initialize an empty DataFrame to store the transactions
    df_transactions = pd.DataFrame()

    # Define the number of transactions to retrieve per API call
    transactions_per_call = 10_000

    # Iterate 'n' times to retrieve 'n' sets of transactions
    for i in range(n):
        # Calculate the start and end blocks for each API call
        start_block = latest_block - (n-i) * transactions_per_call
        end_block = latest_block - (n-1-i) * transactions_per_call
        # print((n-1-i) * transactions_per_call)
        # print(transactions_per_call)
        # print("latest_block",latest_block)
        # print("start_block",start_block)
        # print("end_block",end_block)
        # print()
        # Make the API request
        params = {
            "module": "account",
            "action": "tokentx",
            "contractaddress": contractAddr,
            "startblock": start_block,
            "endblock": end_block,
            "sort": "asc",
            "apikey": api_key
        }
        response = requests.get(url, params=params)

        # Get the transactions data
        transactions = response.json()["result"]

        # Append the transactions to the DataFrame
        df_transactions = pd.concat([df_transactions, pd.DataFrame(transactions)])

        # Add a delay to avoid hitting the API rate limit
        time.sleep(1)

    df_transactions['timeStamp'] = pd.to_datetime(df_transactions['timeStamp'].astype(int), unit='s')
    df_transactions['value'] = df_transactions['value'].astype(float) / 1e18
    return df_transactions


In [3]:
# Define the number of blocks to retrieve transactions from
n_blocks = 20000
n_loop = n_blocks // 10_000
for tokenSymbol, contractAddr in dict_addresses.items():
    file = f"output/transactions_{tokenSymbol}.csv"
    print(file)
    if not os.path.exists(file):
        print("Creating file and scrapping data")
        print(f"Key: {tokenSymbol}, Value: {contractAddr}")
        df_transactions = get_coin_data(tokenSymbol, contractAddr, n_loop)
        # Save transactions to a CSV file
        df_transactions = df_transactions.drop(["confirmations","input"],axis=1)
        df_transactions_no_dup = df_transactions.drop_duplicates(subset="hash")
        df_transactions_no_dup.to_csv(file, sep=";", index=False)
        print(f"All transactions saved to transactions_{tokenSymbol}_.csv")
        print("shape of created transaction dataframe",df_transactions_no_dup.shape)
        print()
    else:
        # UPDATE THE DATA
        print("Completion of data")
        # load file
        df_temp = pd.read_csv(file,sep=";")
        print("shape of existing file ",df_temp.shape)
        # sort descendenly by block number
        df_temp = df_temp.sort_values("blockNumber", ascending=False)
        # get start block for new api call which is last time last block
        start_block = df_temp["blockNumber"].iloc[0]
        # Get the latest block number
        params = {
            "module": "proxy",
            "action": "eth_blockNumber",
            "apikey": api_key
        }
        # api call
        response = requests.get(url, params=params)
        # convert to int
        latest_block = int(response.json()["result"], 16)
        # determine number of loops
        n_loop_to_concat = ((latest_block-start_block)//10000) + 1
        df_transactions = get_coin_data(df_temp["tokenSymbol"].unique(), df_temp["contractAddress"].unique(), n_loop_to_concat)
        #print("shape of latest downloaded df_transactions before merging", df_transactions.shape)
        # Append the transactions to the DataFrame
        df_latest = pd.concat([df_transactions, df_temp]).drop(["confirmations","input"],axis=1)
        df_latest_no_dup = df_latest.drop_duplicates(subset="hash")
        print("shape of latest df_transactions after merging and without duplicates", df_latest_no_dup.shape)
        print("Number of new transactions registered", df_latest_no_dup.shape[0]-df_temp.shape[0])
        print()
        df_latest_no_dup.loc[:,"blockNumber"] = df_latest_no_dup["blockNumber"].astype(int)
        #print(df_latest)
        df_latest_no_dup = df_latest_no_dup.sort_values(by = "blockNumber")
        df_latest_no_dup.to_csv(file, sep=";", index=False)


output/transactions_APE.csv
Completion of data
shape of existing file  (8238, 17)
shape of latest df_transactions after merging and without duplicates (8238, 17)
Number of new transactions registered 0

output/transactions_AXIE.csv
Completion of data
shape of existing file  (1573, 17)
shape of latest df_transactions after merging and without duplicates (1573, 17)
Number of new transactions registered 0

output/transactions_GALA.csv
Completion of data
shape of existing file  (9667, 17)
shape of latest df_transactions after merging and without duplicates (9668, 17)
Number of new transactions registered 1

output/transactions_MANA.csv
Completion of data
shape of existing file  (3826, 17)
shape of latest df_transactions after merging and without duplicates (3826, 17)
Number of new transactions registered 0

output/transactions_ILV.csv
Completion of data
shape of existing file  (1897, 17)
shape of latest df_transactions after merging and without duplicates (1898, 17)
Number of new transacti

In [121]:
df_temp = pd.read_csv("output/transactions_APE.csv",sep=";")
print(df_temp.shape)
#df_temp.drop_duplicates(subset=['confirmations'])


(8213, 17)


## WebApp on jupyter

In [4]:
import os
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Load the data from the CSV files
# Note: You need to replace 'output' with the actual path to your CSV files
dataframes = []
for filename in os.listdir('output'):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join('output', filename), sep=';')
        dataframes.append(df)
df = pd.concat(dataframes)

# Create the Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the app layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1('Token Analysis'),
            dcc.Dropdown(
                id='token-dropdown',
                options=[{'label': i, 'value': i} for i in df['tokenSymbol'].unique()],
                value='MANA'
            ),
            # Add more filters here
        ], width=5),
        dbc.Col([
            dcc.Graph(id='token-graph')
        ], width=7)
    ])
])

# Define the callback to update the graph
@app.callback(
    Output('token-graph', 'figure'),
    [Input('token-dropdown', 'value')]
)
def update_graph(selected_token):
    filtered_df = df[df['tokenSymbol'] == selected_token]
    figure = go.Figure(
        data=[
            go.Scatter(
                x=filtered_df['timeStamp'],
                y=filtered_df['value'],
                mode='lines',
                name='Value over time'
            )
        ],
        layout=go.Layout(
            title='Token Value Over Time',
            yaxis=dict(
                title='Value ('+selected_token+')',  # Change this to 'Value (USD)' if the values are in USD
            ),
            showlegend=True,
            legend=go.layout.Legend(
                x=0,
                y=1.0
            ),
            margin=go.layout.Margin(l=40, r=0, t=40, b=30)
        )
    )
    return figure

# Run app and display result inline in the notebook
app.run_server(mode='inline')


/home/mohcine/mambaforge/envs/ml/lib/python3.10/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



## CMC scrapping

In [3]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from dotenv import load_dotenv
load_dotenv()

url = os.getenv("URL_CMC")
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': os.getenv("API_KEY_CMC"),
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2023-11-09T10:08:39.939Z', 'error_code': 0, 'error_message': None, 'elapsed': 3, 'credit_count': 1, 'notice': None}, 'data': [{'id': 8538, 'name': 'd759ubhb7lu', 'symbol': 'ez7ik92fgno', 'slug': '85z048zx06e', 'cmc_rank': 6699, 'num_market_pairs': 8358, 'circulating_supply': 2698, 'total_supply': 2925, 'max_supply': 332, 'infinite_supply': None, 'last_updated': '2023-11-09T10:08:39.938Z', 'date_added': '2023-11-09T10:08:39.938Z', 'tags': ['b0cmd7k03t', 'mfgn9dk1ag', 'eurd7v0dr4', 'myp1o68m2w', 'zpvgn3dh8xn', '52e0we9kfmk', '4eqz04goay6', 'wdw0l3slex', 'fgo90sgtpqh', 'tsfvhihrejg'], 'platform': None, 'self_reported_circulating_supply': None, 'self_reported_market_cap': None, 'quote': {'USD': {'price': 0.23156208051159188, 'volume_24h': 8576, 'volume_change_24h': 0.2606089871291206, 'percent_change_1h': 0.7714080048551117, 'percent_change_24h': 0.7052731752650081, 'percent_change_7d': 0.35211100235389625, 'market_cap': 0.4779455163332951, 'market_cap_dominance':

In [ ]:
/cryptocurrency/*